In [488]:
import json
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from pandas import json_normalize
from datetime import datetime
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from cfuzzyset import cFuzzySet as FuzzySet

In [533]:
df = pd.read_csv('owl_rock_technology_finance_corp_operations.csv')
df

,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022
0,Investment Income,NaN,NaN
1,"Investment income from non-controlled, non-aff...",NaN,NaN
2,Interest income,120797000.0,8.129200e+07
3,Payment-in-kind interest income,30133000.0,1.717800e+07
4,Dividend income,6339000.0,4.344000e+06
5,Other income,1605000.0,8.750000e+05
6,"Total investment income from non-controlled, n...",158874000.0,1.036890e+08
7,"Investment income from non-controlled, affilia...",NaN,NaN
8,Dividend income,2614000.0,3.799000e+06
9,"Total investment income from non-controlled, a...",2614000.0,3.799000e+06


In [491]:
def clean_text_2 (row):
    no_punc = re.sub(r'[^\w\s]', '', row)
    lower = no_punc.lower()
    no_space = re.sub(r'\s+', '', lower)
    return no_space

def extract_investment_rows (df, start_cell_str, end_cell_str):
    try:
        start_row = df[df.iloc[:, 0].apply(clean_text_2) == start_cell_str].index[0]
        end_row = df[df.iloc[:, 0].apply(clean_text_2) == end_cell_str].index[0]
    except IndexError:
        print("Start or end cell string not found in the DataFrame.")
        return
    portion = df.iloc[start_row + 1 : end_row + 1, :]
    return portion

'hello45'

In [492]:
df_investments = extract_investment_rows(df, 'investmentincome', 'totalinvestmentincome')
df_investments

In [394]:
for i in range(len(df_investments)):
    row = df_investments.iloc[i]
    if pd.isnull(row[1]) and pd.isnull(row[2]):
        print(clean_text_2(row[0]))
        # print(row[0])

In [494]:
def top_score (row, terms):
    scores = []
    standard_row = clean_text_2 (row)
    for search_string in terms:
        standard_term = clean_text_2 (search_string)
        score = fuzz.partial_token_set_ratio (standard_row, standard_term)
        # if standard_row in standard_term:
        #     score += 10
        scores.append(score)
    if score >= 45:
        return max(scores)
    else: return 0

In [495]:
def best_match (string, dict):
    best_match, best_match_type, best_match_score = '', '', 0
    for key in dict:
        score = top_score (string, key['terms'])
        if score > best_match_score:
            best_match, best_match_type, best_match_score = key['category'], key['type'], score
    if best_match_score <= 60:
        best_match, best_match_type = 'No match', 'No match'
    return best_match, best_match_type, best_match_score

In [501]:
def generate_headings (df):
    new_df = df.copy()
    headings = []
    current_heading, current_heading_type = 'No match', 'investment'
    for i in range(len(df)):
        row = df.iloc[i]
        if pd.isnull(row[1]) and pd.isnull(row[2]):
           heading_guess, heading_guess_type, _ = best_match (row[0], investment_categories + income_categories)
           # heading_guess, heading_guess_type, _ = best_match (find_income(row[0]), income_categories)
           current_heading, current_heading_type = heading_guess, heading_guess_type
        headings.append((current_heading, current_heading_type))
    new_df['headings'] = headings
    return new_df

In [549]:
def generate_subheadings (df):
    new_df = df.copy()
    subheadings = []
    for i in range(len(df)):
        row = df.iloc[i]
        _, current_heading_type = row['headings']
        if not(pd.isnull(row[1]) and pd.isnull(row[2])):
           if current_heading_type == 'investment':
               subheading_guess, subheading_guess_type, _ = best_match (find_income(row[0]), income_categories)
           if current_heading_type == 'income':
                subheading_guess, subheading_guess_type, _ = best_match (row[0], investment_categories)
           subheadings.append((subheading_guess, subheading_guess_type))
        else:
            subheadings.append(('No match', 'investment'))
    new_df['subheadings'] = subheadings
    new_df.iloc[-1]['headings'] = ('total', 'investment')
    return new_df
    

In [554]:
# def fix_last_row (df):
#     df.iloc[-1]['headings'] = ('total', 'investment')
#     print(df.iloc[-1]['subheadings'])
#     return df

# fix this function since it gets an error since i'm setting the value of a copy
def fix_last_row (df):
    df.iloc[-1]['headings'] = ('total', 'investment')
    df.iloc[-1]['subheadings'] = ('total', 'income')


    # new_df.iloc[-1]['headings'] = ('total', 'investment')
    # new_df.iloc[-1]['subheadings'] = ('total', 'income')
    # return new_df

# Change last row of Dataframe to ('total', 'investment') and ('total', 'income') for the column 'headings' and 'subheadings' respectively
fix_last_row(test_df)
test_df

/var/folders/g_/cwvjv87n3wq590ldrjtpds000000gn/T/ipykernel_6073/878857854.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[-1]['headings'] = ('total', 'investment')
/var/folders/g_/cwvjv87n3wq590ldrjtpds000000gn/T/ipykernel_6073/878857854.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[-1]['subheadings'] = ('total', 'income')


,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022,headings,subheadings
1,"Investment income from non-controlled, non-aff...",NaN,NaN,"(non_affiliated, investment)","(No match, investment)"
2,Interest income,120797000.0,81292000.0,"(non_affiliated, investment)","(interest, income)"
3,Payment-in-kind interest income,30133000.0,17178000.0,"(non_affiliated, investment)","(pik, income)"
4,Dividend income,6339000.0,4344000.0,"(non_affiliated, investment)","(dividend, income)"
5,Other income,1605000.0,875000.0,"(non_affiliated, investment)","(other, income)"
6,"Total investment income from non-controlled, n...",158874000.0,103689000.0,"(non_affiliated, investment)","(total, income)"
7,"Investment income from non-controlled, affilia...",NaN,NaN,"(affiliated, investment)","(No match, investment)"
8,Dividend income,2614000.0,3799000.0,"(affiliated, investment)","(dividend, income)"
9,"Total investment income from non-controlled, a...",2614000.0,3799000.0,"(affiliated, investment)","(total, income)"
10,Total Investment Income,161488000.0,107488000.0,"(affiliated, investment)","(total, income)"


In [456]:
fuzz.token_set_ratio ('interestincome', 'total')

21

In [479]:
# Look for "income" in a string and return all characters before it
def find_income (row):
    if 'income' in row:
        return row.split('income')[0]
    else: return row

In [484]:
best_match(find_income(clean_text_2("interest, fee, and dividend income")), income_categories)
find_income(clean_text_2("interest, fee, and dividend income"))
# fuzz.partial_token_set_ratio(find_income(clean_text_2("interest, fee, and dividend income")), 'dividend')
# top_score (find_income(clean_text_2("interest, fee, and dividend income")), ['dividend'])

'interestfeeanddividend'

In [457]:
find_income ("investmentincome")

'investment'

In [550]:
test_df = generate_subheadings (generate_headings (df_investments))
test_df

/var/folders/g_/cwvjv87n3wq590ldrjtpds000000gn/T/ipykernel_6073/3534848080.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.iloc[-1]['headings'] = ('total', 'investment')


,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022,headings,subheadings
1,"Investment income from non-controlled, non-aff...",NaN,NaN,"(non_affiliated, investment)","(No match, investment)"
2,Interest income,120797000.0,81292000.0,"(non_affiliated, investment)","(interest, income)"
3,Payment-in-kind interest income,30133000.0,17178000.0,"(non_affiliated, investment)","(pik, income)"
4,Dividend income,6339000.0,4344000.0,"(non_affiliated, investment)","(dividend, income)"
5,Other income,1605000.0,875000.0,"(non_affiliated, investment)","(other, income)"
6,"Total investment income from non-controlled, n...",158874000.0,103689000.0,"(non_affiliated, investment)","(total, income)"
7,"Investment income from non-controlled, affilia...",NaN,NaN,"(affiliated, investment)","(No match, investment)"
8,Dividend income,2614000.0,3799000.0,"(affiliated, investment)","(dividend, income)"
9,"Total investment income from non-controlled, a...",2614000.0,3799000.0,"(affiliated, investment)","(total, income)"
10,Total Investment Income,161488000.0,107488000.0,"(affiliated, investment)","(total, income)"


In [180]:
best_match ('totalinvestmentincomefromnoncontrollednonaffiliatedinvestments', investment_categories)

('non_affiliated', 'investment', 100)

In [167]:
top_score ('totalinvestmentincomefromnoncontrollednonaffiliatedinvestments', ['interestincomeexcluding'])
# income_categories[1]['terms']

# Note that if the top score is 100, then move on

100

In [512]:

# capitalstructuringservicefees

# otherincome
# totalinvestmentincomefromnoncontrollednonaffiliatecompanyinvestments
# interestincomeexcludingpikinterestincome
# pikinterestincome
# totalinvestmentincomefromnoncontrolledaffiliatecompanyinvestments
# interestincomeexcludingpikinterestincome
# pikinterestincome
# capitalstructuringservicefees
# dividendincome
# otherincome
# totalinvestmentincomefromcontrolledaffiliatecompanyinvestments
# totalinvestmentincome

investment_categories = [
    {
        'category': 'non_affiliated',
        'type': 'investment',
        'terms': ['noncontrolledunaffiliatedinvestments', 
                  'noncontrollednonaffiliatecompanyinvestments',
                  'noncontrollednonaffiliatedinvestments',
                  'noncontrolnonaffiliateinvestments']
    },
    {
        'category': 'affiliated',
        'type': 'investment',
        'terms': ['noncontrolledaffiliatedinvestments', 
                  'noncontrolledaffiliatecompanyinvestments',
                  'noncontrolledaffiliatedinvestments',
                  'affiliateinvestments']
    },
    {
        'category': 'control',
        'type': 'investment',
        'terms': ['controlledaffiliatedinvestments',
                  'controlledaffiliatecompanyinvestments',
                  'controlledaffiliatedinvestments',
                  'controlinvestments']
    },
    ]

income_categories = [
    {
        'category': 'interest',
        'type': 'income',
        'terms': ['interest',
                  'incomeexcluding']
    },
    {
        'category': 'total',
        'type': 'income',
        'terms': ['total', 
                  'interestfeeanddividend',
                  'totalinvestment']
    },
    {
        'category': 'dividend',
        'type': 'income',
        'terms': ['dividend']
    },
    { 
        'category': 'pik',
        'type': 'income',
        'terms': ['pik', 'paymentinkind', 'paidinkind']
    },
    {
        'category': 'fee',
        'type': 'income',
        'terms': ['fee']
    },
    {
        'category': 'other',
        'type': 'income',
        'terms': ['other']
    }
]


affiliated_terms = ['noncontrolledaffiliatedinvestments', 
                    'noncontrolledaffiliatecompanyinvestments',
                    'noncontrolledaffiliatedinvestments']

control_terms = ['controlledaffiliatedinvestments',
                 'controlledaffiliatecompanyinvestments',
                 'controlledaffiliatedinvestments']

ifd_income_terms = ['interestfeeanddividendincome']

In [81]:
interest_income_terms = ['interest income']
dividend_income_terms = ['dividend income']
pik_income_terms = ['payment in kind interest income', 'PIK interest income']
other_income_terms = ['other income', 'fee income']
total_income_terms = ['total investment income']


# non_affiliated_terms = ['non-control, non-affiliate',
#                         'non-control, non-affiliated',
#                         'non-controlled, non-affiliate',
#                         'non-controlled, non-affiliated']


# affiliated_terms = ['non-control, affiliate',
#                     'non-control, affiliated',
#                     'non-controlled, affiliate',
#                     'non-controlled, affiliated']


# control_terms = ['control, affiliate',
#                  'control, affiliated',
#                  'controlled, affiliate',
#                  'controlled, affiliated']

investments_matrix = [
    {
        'affiliation': 'Header',
        'affiliation_terms': ['Header'],
        'income_type': 'Investment income',
        'income_type_terms': ['Header'],
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'None',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'Interest',
        'income_type_terms': interest_income_terms,
        'value1': None,
        'value2': None
    },
    {   'affiliation': 'None',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'PIK',
        'income_type_terms': pik_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'None',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'Dividend',
        'income_type_terms': dividend_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'None',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'Other',
        'income_type_terms': other_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'None',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'IFD',
        'income_type_terms': ifd_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'None',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'Total',
        'income_type_terms': total_income_terms,
        'value1': None,
        'value2': None
    },



    {
        'affiliation': 'Affiliated',
        'affiliation_terms': affiliated_terms,
        'income_type': 'Interest',
        'income_type_terms': interest_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Affiliated',
        'affiliation_terms': affiliated_terms,
        'income_type': 'PIK',
        'income_type_terms': pik_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Affiliated',
        'affiliation_terms': affiliated_terms,
        'income_type': 'Dividend',
        'income_type_terms': dividend_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Affiliated',
        'affiliation_terms': affiliated_terms,
        'income_type': 'Other',
        'income_type_terms': other_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Affiliated',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'IFD',
        'income_type_terms': ifd_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Affiliated',
        'affiliation_terms': affiliated_terms,
        'income_type': 'Total',
        'income_type_terms': total_income_terms,
        'value1': None,
        'value2': None
    },



    {
        'affiliation': 'Controlled',
        'affiliation_terms': control_terms,
        'income_type': 'Interest',
        'income_type_terms': interest_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Controlled',
        'affiliation_terms': control_terms,
        'income_type': 'PIK',
        'income_type_terms': pik_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Controlled',
        'affiliation_terms': control_terms,
        'income_type': 'Dividend',
        'income_type_terms': dividend_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Controlled',
        'affiliation_terms': control_terms,
        'income_type': 'Other',
        'income_type_terms': other_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Controlled',
        'affiliation_terms': non_affiliated_terms,
        'income_type': 'IFD',
        'income_type_terms': ifd_income_terms,
        'value1': None,
        'value2': None
    },
    {
        'affiliation': 'Controlled',
        'affiliation_terms': control_terms,
        'income_type': 'Total',
        'income_type_terms': total_income_terms,
        'value1': None,
        'value2': None
    }
]

In [301]:
def return_highest_score_match (text, type_list):
    text = clean_text_2(text)
    highest_average = 0
    winner = pd.NA
    for header in type_list:
        score = top_score(text, header['search_strings'])
        if score > highest_average:
            highest_average = score
            winner = header['row_title']
    for header in type_list:
        for search_string in header['search_strings']:
            if search_string == text or search_string in text:
                winner = header['row_title']
    return winner

In [302]:
def best_match_subheading (subheading, matrix):
    best_match = '---'
    best_match_score = 0
    for row in matrix:
        affl_avg_score = top_score (subheading, row['affiliation_terms'])
        if affl_avg_score > best_match_score:
            best_match, best_match_type = row['affiliation'], 'affiliation'
            best_match_score = affl_avg_score
        for term in row['affiliation_terms']:
            if term == subheading:
                return row['affiliation'], 'affiliation'
            
        inv_avg_score = top_score (subheading, row['income_type_terms'])
        if inv_avg_score > best_match_score:
            best_match, best_match_type = row['income_type'], 'income_type'
            best_match_score = inv_avg_score
        for term in row['income_type_terms']:
            if term == subheading:
                return row['income_type'], 'income_type'
    if best_match_score >= 60:
        return best_match, best_match_type
    else:
        return '---', 'affiliation'

In [303]:
def best_match_text (text, matrix, subheading_match, subheading_type):
    dicts = [d for d in matrix if d[subheading_type] == subheading_match]
    best_match = '---'
    best_match_score = 0
    if subheading_type == 'affiliation':
        for d in dicts:
            score = top_score (text, d['income_type_terms'])
            if score > best_match_score:
                best_match = d['income_type']
                best_match_score = score
                best_match_type = 'income_type'
    if subheading_type == 'income_type':
        for d in dicts:
            score = top_score (text, d['affiliation_terms'])
            if score > best_match_score:
                best_match = d['affiliation']
                best_match_score = score
                best_match_type = 'affiliation'
    if best_match_score >= 60:
        return best_match, best_match_type
    else: return '---', 'affiliation'

In [308]:
df_investments

,Unnamed: 0,Three Months Ended 2023,Three Months Ended 2022
0,INVESTMENT INCOME:,NaN,NaN
1,From non-controlled/non-affiliate company inve...,NaN,NaN
2,Interest income (excluding payment-in-kind (“P...,367000000.0,242000000.0
3,PIK interest income,37000000.0,27000000.0
4,Capital structuring service fees,10000000.0,26000000.0
5,Dividend income,57000000.0,41000000.0
6,Other income,16000000.0,10000000.0
7,Total investment income from non-controlled/no...,487000000.0,346000000.0
8,From non-controlled affiliate company investme...,NaN,NaN
9,Interest income (excluding PIK interest income),3000000.0,1000000.0


In [315]:
top_score('PIK interest income', ['interest income'])

100

In [313]:
best_match_text ('PIK interest income',
                 investments_matrix, 'Affiliated', 'affiliation')

('Interest', 'income_type')

In [305]:
top_score ('Total investment income from non-controlled/non-affiliate company investments', affiliated_terms)

87

In [306]:
for i in range(len(df_investments)):
    row = df_investments.iloc[i]
    print(best_match_subheading (row[0], investments_matrix))

('Total', 'income_type')
('None', 'affiliation')
('Interest', 'income_type')
('PIK', 'income_type')
('---', 'affiliation')
('Dividend', 'income_type')
('Other', 'income_type')
('None', 'affiliation')
('Affiliated', 'affiliation')
('Interest', 'income_type')
('PIK', 'income_type')
('Total', 'income_type')
('Controlled', 'affiliation')
('Interest', 'income_type')
('PIK', 'income_type')
('---', 'affiliation')
('Dividend', 'income_type')
('Other', 'income_type')
('Total', 'income_type')
('Total', 'income_type')


In [307]:
def get_investment (df, matrix):
    new_matrix = matrix.copy()
    subheading_match, subheading_type = '', ''
    for i in range(len(df)):
        row = df.iloc[i]
        if pd.isnull(row[1]) and pd.isnull(row[2]):
            subheading_match, subheading_type = best_match_subheading(row[0], matrix)
            continue
        text_match, text_type = best_match_text(row[0], matrix, subheading_match, subheading_type)
        print(i, subheading_match, text_match)
        for item in new_matrix:
           if item[text_type] == text_match and item[subheading_type] == subheading_match:
              print('match found')
        #         # print(item)
        #         # new_matrix.append(item)
        #         item['value1'] = row[1]
        #         item['value2'] = row[2]
    print(new_matrix)
    # return new_matrix

In [309]:
get_investment(df_investments, investments_matrix)


2 None Interest
match found
3 None Interest
match found
4 None ---
5 None Dividend
match found
6 None Other
match found
7 None Total
match found
9 Affiliated Interest
match found
10 Affiliated Interest
match found
11 Affiliated Total
match found
13 Controlled Interest
match found
14 Controlled Interest
match found
15 Controlled ---
16 Controlled Dividend
match found
17 Controlled Other
match found
18 Controlled Total
match found
19 Controlled Total
match found
[{'affiliation': 'Header', 'affiliation_terms': ['Header'], 'income_type': 'Investment income', 'income_type_terms': ['Header'], 'value1': None, 'value2': None}, {'affiliation': 'None', 'affiliation_terms': ['non-control, non-affiliate', 'non-control, non-affiliated', 'non-controlled, non-affiliate', 'non-controlled, non-affiliated'], 'income_type': 'Interest', 'income_type_terms': ['interest income'], 'value1': None, 'value2': None}, {'affiliation': 'None', 'affiliation_terms': ['non-control, non-affiliate', 'non-control, non-af